In [55]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import pyarrow

In [9]:
#!pip install pyarrow

In [3]:
cwd = os.getcwd()
cwd

'/Users/michelle/MIDS/210 Capstone'

In [26]:
joined = pd.read_parquet('joined_datetime_ym.parquet', engine='pyarrow')

income = pd.read_csv('income_data.csv', low_memory=False)


In [57]:
joined_income = pd.read_parquet('joined_datetime_ym_with_income.parquet', engine='pyarrow')

In [59]:
joined_income.dtypes

year                                float64
cdscode                              object
school_county                        object
school_zip                            int64
school_region_name                   object
school_open_date             datetime64[ns]
school_closed_date           datetime64[ns]
school_lat                          float64
school_lon                          float64
school_wspd                         float64
school_elevation_m                  float64
year_month                           object
total_population                     object
total_population_male                object
total_population_female              object
population_0_4                       object
population_0_4_male                  object
population_0_4_female                object
population_5_9                       object
population_5_9_male                  object
population_5_9_female                object
population_10_14                     object
population_10_14_male           

In [98]:
#elevations.head()

In [27]:
def str_to_int(zip_code):
    zip_code = zip_code
    try:
        zip_code = int(zip_code)
    except:
        print(zip_code)
    return zip_code

In [99]:
#income[income['Zip Code']=='Unallocated']

# Dropping rows where income dataset doesn't have zip codes
Can't supplement; dropping

In [28]:
idx = income[ (income['Zip Code'] == 'Unallocated') 
                  | (income['Zip Code'] == 'Suppressed') 
                 | (income['Zip Code'] == 'Suppressed1')
                 | (income['Zip Code'] == 'Unallocated2')].index
income.drop(idx , inplace=True)

In [29]:
# checking
income[income['Zip Code']=='Unallocated']

,Taxable Year,Zip Code,State,City,County,Returns,CA AGI,Total Tax Liability,CountyLatitude,CountyLongitude,Geo County,Geo City,Geo ZipCode


In [30]:
# converting zip code column to ints
income['Zip Code'] = income['Zip Code'].apply(str_to_int)

In [31]:
# get unique zips from income dataset
unique_income_zips = set(income['Zip Code'].unique())
len(unique_income_zips)

2534

In [32]:
# converting schools joined dataset zip code column to ints
joined['school_zip'] = joined['school_zip'].apply(str_to_int)

In [33]:
# get unique zips from schools dataset
unique_school_zips = set(joined['school_zip'].unique())
len(unique_school_zips)

1539

In [34]:
print("{} unique income dataset zips".format(len(unique_income_zips)))
print("{} unique school zips".format(len(unique_school_zips)))

2534 unique income dataset zips
1539 unique school zips


In [35]:
# getting number of same unique zip codes across both datasets
intersect_zips = unique_income_zips.intersection(unique_school_zips)
len(intersect_zips)

1527

# Check each year of income dataset for # unique zip codes

In [36]:
income['Taxable Year'].value_counts().sort_index(ascending=True)

1992    2314
1993    2311
1994    2298
1995    2365
1996    2349
1997    2354
1998    2360
1999    2355
2000    2362
2001    2364
2002    2373
2003    2385
2004    2390
2005    2404
2006    2377
2007    2373
2008    2384
2009    2380
2010    2380
2011    2371
2012    2368
2013    2365
2014    2364
2015    2378
2016    2365
2017    2362
2018    2361
2019    2360
2020    2358
Name: Taxable Year, dtype: int64

In [37]:
years = list(income['Taxable Year'].unique())
len(years)

29

In [38]:
# make dict of unique zip codes per year
list_unique_zips_per_year = {}
for i in years:
    temp = income[income['Taxable Year'] == i]
    temp_unique_zips = set(temp['Zip Code'].unique())
    list_unique_zips_per_year[i] = temp_unique_zips

In [74]:
#list_unique_zips_per_year

In [39]:
len(list_unique_zips_per_year)

29

In [40]:
# function to get the count and list of intersecting zip codes between unique school zips
# and unique zips per year
def check_zip_intersect(unique_school_zips, year, unique_income_zips_year):
    unique_school_zips = unique_school_zips
    unique_income_zips_year = unique_income_zips_year
    year = year
    intersect_zips = unique_income_zips_year.intersection(unique_school_zips)
    #print("Number of intersecting zip codes for year {}:\n{}\n\n".format(year, len(intersect_zips)))
    return len(intersect_zips), intersect_zips

In [41]:
# dumping into dictionaries:
# key is year; value is the number of zip codes that intersect
year_and_count_intersect_zips = {}
# key is year; value is the list of zip codes in the intersect
year_and_list_intersect_zips = {}
for k, v in list_unique_zips_per_year.items():
    year_and_count_intersect_zips[k], year_and_list_intersect_zips[k] = check_zip_intersect(unique_school_zips, k, v)



In [42]:
year_and_count_intersect_zips

{2015: 1527,
 2003: 1518,
 2017: 1525,
 2004: 1520,
 2007: 1517,
 1995: 1488,
 2010: 1525,
 1992: 1446,
 2019: 1524,
 2001: 1511,
 2018: 1525,
 1998: 1501,
 2014: 1525,
 2016: 1524,
 2009: 1525,
 1994: 1459,
 2006: 1517,
 2012: 1524,
 2013: 1524,
 2020: 1525,
 2011: 1526,
 2002: 1513,
 2008: 1525,
 1996: 1490,
 2000: 1506,
 1999: 1503,
 1997: 1493,
 2005: 1523,
 1993: 1458}

In [43]:
# filter to years 2002-2017
filtered_counts = {key: year_and_count_intersect_zips[key] for key in range(2002,2018)}
filtered_counts

{2002: 1513,
 2003: 1518,
 2004: 1520,
 2005: 1523,
 2006: 1517,
 2007: 1517,
 2008: 1525,
 2009: 1525,
 2010: 1525,
 2011: 1526,
 2012: 1524,
 2013: 1524,
 2014: 1525,
 2015: 1527,
 2016: 1524,
 2017: 1525}

In [44]:
#joined['year_and_list_intersect_zips']


In [45]:
joined.head()

,year,cdscode,school_county,school_zip,school_region_name,school_open_date,school_closed_date,school_lat,school_lon,school_wspd,...,school_wdir_wrt_0N,ps_wdir_wrt_0N,school_wind_alignment,ps_wind_alignment,avg_wind_speed,avg_wind_alignment,avg_wind_alignment_cosine,new_year,new_ym,open_close_dates_valid
0,2010.0,20102070109728,Madera,93610,Northern San Joaquin Valley,2005-08-16,2014-06-30,37.115210,-120.266030,1.531352,...,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,2010-04-01,1
1,2010.0,20102072030187,Madera,93610,Northern San Joaquin Valley,2001-01-22,2014-06-30,37.115210,-120.266030,1.531352,...,102.528057,102.528057,157.824497,157.824497,1.531352,157.824497,0.073968,2010-01-01,2010-04-01,1
2,2010.0,20651930000000,Madera,93610,Northern San Joaquin Valley,NaT,NaT,37.122449,-120.264870,1.531352,...,102.528057,102.528057,140.068342,140.068342,1.531352,140.068342,0.233189,2010-01-01,2010-04-01,0
3,2010.0,20651930113076,Madera,93610,Northern San Joaquin Valley,2007-08-21,NaT,37.131113,-120.242025,1.531352,...,102.528057,102.528057,90.696835,90.696835,1.531352,90.696835,0.987838,2010-01-01,2010-04-01,2
4,2010.0,20651936023907,Madera,93610,Northern San Joaquin Valley,1980-07-01,NaT,37.115243,-120.267770,1.531352,...,102.528057,102.528057,160.255014,160.255014,1.531352,160.255014,0.058794,2010-01-01,2010-04-01,2


In [46]:
income['CA_AGI_per_returns'] = income['CA AGI']/income['Returns']

In [47]:
income.head()

,Taxable Year,Zip Code,State,City,County,Returns,CA AGI,Total Tax Liability,CountyLatitude,CountyLongitude,Geo County,Geo City,Geo ZipCode,CA_AGI_per_returns
0,2015,91101,CA,Pasadena,Los Angeles,9841,1000029587,75705318,34.059483,-118.242048,POINT (-118.242048 34.0594833),POINT (-118.141452 34.14955),POINT (-118.139043 34.14764),101618.695966
1,2015,95667,CA,Placerville,El Dorado,16307,1000097894,36234728,38.763942,-120.606059,POINT (-120.6060593 38.76394197),POINT (-120.799147 38.729652),POINT (-120.794085 38.726337),61329.361256
2,2003,95446,CA,Guerneville,Sonoma,2357,100012444,3631836,38.493672,-122.930263,POINT (-122.9302632 38.49367199),POINT (-122.998768 38.501992),POINT (-122.996796 38.510032),42432.093339
3,2017,92007,CA,Cardiff By The Sea,San Diego,5909,1000162607,81875263,32.789640,-117.193658,POINT (-117.1936577 32.78964014),POINT (-117.280296 33.021702),POINT (-117.275252 33.023512),169260.891352
4,2004,90280,CA,South Gate,Los Angeles,35211,1000239478,12188238,34.059483,-118.242048,POINT (-118.242048 34.0594833),POINT (-118.206087 33.954211),POINT (-118.19452 33.943676),28407.017069


In [48]:
income_subset = income[["Taxable Year", "Zip Code", "CA_AGI_per_returns", "Total Tax Liability"]]

In [49]:
joined_with_income = pd.merge(joined, income_subset,  how='left', left_on=['year', 'school_zip'], right_on = ['Taxable Year', 'Zip Code'])

In [50]:
joined_with_income.head()

,year,cdscode,school_county,school_zip,school_region_name,school_open_date,school_closed_date,school_lat,school_lon,school_wspd,...,avg_wind_speed,avg_wind_alignment,avg_wind_alignment_cosine,new_year,new_ym,open_close_dates_valid,Taxable Year,Zip Code,CA_AGI_per_returns,Total Tax Liability
0,2010.0,20102070109728,Madera,93610,Northern San Joaquin Valley,2005-08-16,2014-06-30,37.115210,-120.266030,1.531352,...,1.531352,157.824497,0.073968,2010-01-01,2010-04-01,1,2010.0,93610.0,14375.028223,2905021.0
1,2010.0,20102072030187,Madera,93610,Northern San Joaquin Valley,2001-01-22,2014-06-30,37.115210,-120.266030,1.531352,...,1.531352,157.824497,0.073968,2010-01-01,2010-04-01,1,2010.0,93610.0,14375.028223,2905021.0
2,2010.0,20651930000000,Madera,93610,Northern San Joaquin Valley,NaT,NaT,37.122449,-120.264870,1.531352,...,1.531352,140.068342,0.233189,2010-01-01,2010-04-01,0,2010.0,93610.0,14375.028223,2905021.0
3,2010.0,20651930113076,Madera,93610,Northern San Joaquin Valley,2007-08-21,NaT,37.131113,-120.242025,1.531352,...,1.531352,90.696835,0.987838,2010-01-01,2010-04-01,2,2010.0,93610.0,14375.028223,2905021.0
4,2010.0,20651936023907,Madera,93610,Northern San Joaquin Valley,1980-07-01,NaT,37.115243,-120.267770,1.531352,...,1.531352,160.255014,0.058794,2010-01-01,2010-04-01,2,2010.0,93610.0,14375.028223,2905021.0


In [51]:
joined_with_income['CA_AGI_per_returns'].isnull().sum()

6367

In [52]:
len(joined_with_income)

2983111

In [53]:
joined_with_income.to_parquet("joined_datetime_ym_with_income.parquet")